In [ ]:
import sys

In [ ]:
import os

In [ ]:
from pathlib import Path

In [ ]:
project_dir = Path(__name__).resolve().parents[1]

In [ ]:
sys.path.insert(0, str(project_dir/'src'/'data'))

In [ ]:
from scrape_bol import *

In [ ]:
from create_products import *

In [ ]:
from datetime import datetime

In [ ]:
s = SearchTerm(search_term='philips sonicare', search_domain = 'bol.com', max_pages = 1)

In [ ]:
len(s.productlist)

In [ ]:
print(s.Scraper.page)

In [ ]:
dir(s)

In [ ]:
datetime.now().strftime("%Y-%m-%d %I:%M%p")

In [ ]:
to_save_data = ['\t'.join([p.provider if p.provider else '', 
                          p.provider_id if p.provider_id else '', 
                           p.name  if p.name else '', 
                           p.price  if p.price else '', 
                           p.original_price  if p.original_price else '' ]) 
                for p in s.productlist]

In [ ]:
with open("../data/raw/price_tracker.tsv", 'a') as file:
    file.writelines(to_save_data)

In [ ]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?searchtext=philips+sonicare')

In [ ]:
pl = bol.get_productlist()

In [ ]:
bol = BolScraper('https://www.bol.com/nl/nl/s/?page=8&searchtext=philips+sonicare&view=list') # No products

In [ ]:
pl = bol.get_productlist()

In [ ]:
pl

In [ ]:
# pl

In [ ]:
from pathlib import Path

In [ ]:
project_dir = Path(__name__).resolve().parents[2]

In [ ]:
project_dir/'data'/'raw'/'example.csv'

# Create DB

In [ ]:
import sqlite3
from sqlite3 import Error

In [ ]:
conn = sqlite3.connect("../data/raw/price_tracker.db",  timeout=10)

In [ ]:
sql_create_products_table = """ CREATE TABLE IF NOT EXISTS projects (
                                    id integer PRIMARY KEY,
                                    name text NOT NULL,
                                    begin_date text,
                                    end_date text
                                ); """

sql_create_prices_table = """CREATE TABLE IF NOT EXISTS prices (
                                datetime datetime NOT NULL,
                                provider text ,
                                provider_id text,
                                name text,
                                price numeric,
                                original_price numeric
                            );"""

In [ ]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
create_table(conn, sql_create_prices_table)

In [ ]:
conn.close()

# Pandas

In [125]:
import pandas as pd

data_dir = project_dir/'data'/'raw'

full_list = list(data_dir.glob('*.tsv'))

max_file = max(full_list, key = lambda x: x.stat().st_mtime )

partial_list = full_list.copy()
partial_list.remove(max_file)

second_max_file = max(partial_list, key = lambda x: x.stat().st_mtime )

second_max_file

df_max = pd.read_csv(max_file, sep = '\t')

df_second_max = pd.read_csv(second_max_file, sep = '\t')

df = df_second_max.compare(df_max)

df = pd.merge(df_second_max, df_max, left_on = ['provider_id'], right_on = ['provider_id'], 
        suffixes = (None, '_old'))

df['discount'] = 1 - df['price'] / df['price_old']

df_notify = df[df['discount'] >=0.05]

cols = ['provider', 'name', 'price', 'price_old', 'URL']

if len(df_notify)>0:
    records = df_notify[cols].to_records(index = False)

In [153]:
df_notify

,provider,provider_id,name,price,original_price,URL,provider_old,name_old,price_old,original_price_old,URL_old,discount
0,bol,9200000055111100,Philips Sonicare for kids HX6321/03 - Elektris...,31.99,"64,99",https://bol.com/nl/nl/p/philips-sonicare-for-k...,bol,Philips Sonicare for kids HX6321/03 - Elektris...,39.95,NaN,https://bol.com/nl/nl/p/philips-sonicare-for-k...,0.199249
4,bol,9300000000290851,Philips Sonicare DiamondClean 9000 HX9914/55 -...,159.99,"329,99",https://bol.com/nl/nl/p/philips-sonicare-diamo...,bol,Philips Sonicare DiamondClean 9000 HX9914/55 -...,209.15,NaN,https://bol.com/nl/nl/p/philips-sonicare-diamo...,0.235047
6,bol,9200000119412119,Philips Sonicare ProtectiveClean 4300 HX6809/3...,92.99,"149,99",https://bol.com/nl/nl/p/philips-sonicare-prote...,bol,Philips Sonicare ProtectiveClean 4300 HX6809/3...,118.99,NaN,https://bol.com/nl/nl/p/philips-sonicare-prote...,0.218506
8,bol,9300000005081673,Philips Sonicare For Kids HX6352/42 - Elektris...,34.99,"49,99",https://bol.com/nl/nl/p/philips-sonicare-for-k...,bol,Philips Sonicare For Kids HX6352/42 - Elektris...,39.95,NaN,https://bol.com/nl/nl/p/philips-sonicare-for-k...,0.124155
9,bol,9200000079522243,Philips Sonicare DiamondClean Smart HX9924/13 ...,169.99,"309,99",https://bol.com/nl/nl/p/philips-sonicare-diamo...,bol,Philips Sonicare DiamondClean Smart HX9924/13 ...,209.00,NaN,https://bol.com/nl/nl/p/philips-sonicare-diamo...,0.186651
12,bol,9200000079522241,Philips Sonicare DiamondClean Smart HX9954/53 ...,184.99,"339,99",https://bol.com/nl/nl/p/philips-sonicare-diamo...,bol,Philips Sonicare DiamondClean Smart HX9954/53 ...,228.99,NaN,https://bol.com/nl/nl/p/philips-sonicare-diamo...,0.192148
19,bol,9200000093063171,Philips Sonicare ProtectiveClean 4500 HX6833/2...,82.99,NaN,https://bol.com/nl/nl/p/philips-sonicare-prote...,bol,Philips Sonicare ProtectiveClean 4500 HX6833/2...,99.99,NaN,https://bol.com/nl/nl/p/philips-sonicare-prote...,0.170017
25,bol,9200000078316283,BRITA fill&enjoy Marella Cool Graphite Half Ye...,49.95,NaN,https://bol.com/nl/nl/p/brita-fill-enjoy-marel...,bol,BRITA fill&enjoy Marella Cool Graphite Half Ye...,52.94,NaN,https://bol.com/nl/nl/p/brita-fill-enjoy-marel...,0.056479


In [141]:
record = records[0]

In [151]:
records = df_notify.to_dict(orient = 'records')
r = []
for record in records:
    price = record['price']
    original_price = record['price_old']
    url = record['URL']

    r.append('\n'.join([record['provider'], record['name'], f'{price}~{original_price}~',
                            f'<{url}|Product Link>'
                           ])
            )

In [152]:
'\n\n'.join(r)

'bol\nPhilips Sonicare for kids HX6321/03 - Elektrische Tandenborstel - Blauw\n31.99~39.95~\n<https://bol.com/nl/nl/p/philips-sonicare-for-kids-hx6321-03-elektrische-tandenborstel-blauw/9200000055111100/|Product Link>\n\nbol\nPhilips Sonicare DiamondClean 9000 HX9914/55 - Elektrische tandenborstel - Wit - 2 stuks\n159.99~209.15~\n<https://bol.com/nl/nl/p/philips-sonicare-diamondclean-9000-hx9914-55-elektrische-tandenborstel-wit-2-stuks/9300000000290851/|Product Link>\n\nbol\nPhilips Sonicare ProtectiveClean 4300 HX6809/35 Duo - Elektrische tandenborstel - Wit\n92.99~118.99~\n<https://bol.com/nl/nl/p/philips-sonicare-protectiveclean-4300-hx6809-35-duo-elektrische-tandenborstel-wit/9200000119412119/|Product Link>\n\nbol\nPhilips Sonicare For Kids HX6352/42 - Elektrische tandenborstel - roze\n34.99~39.95~\n<https://bol.com/nl/nl/p/philips-sonicare-for-kids-hx6352-42-elektrische-tandenborstel-roze/9300000005081673/|Product Link>\n\nbol\nPhilips Sonicare DiamondClean Smart HX9924/13 - Elekt

# Dotenv

In [ ]:
from dotenv import find_dotenv, load_dotenv

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
f = project_dir.parent / 'Utils'

In [ ]:
f